<h2 style="text-align:center;color:#0F4C81;">
Backpropagation
</h2>

## Short introduction to PyTorch autograd

**Example 1**

$$
a = 1.0 \\
b = 2.0 \\
c = a^2 + \sqrt{b}
$$

$$
\frac{\partial c}{\partial a} = 2a = 2 \cdot 1.0 = 2.0 \\
\frac{\partial c}{\partial b} = \frac{1}{2\sqrt{b}} = \frac{1}{2\cdot \sqrt{2}} = 0.3536
$$

In [38]:
import torch

a = torch.tensor(1., requires_grad=True)
b = torch.tensor(2., requires_grad=True)
c = a ** 2 + b ** 0.5

In [39]:
print(f'{a.grad=}, {b.grad=}')

a.grad=None, b.grad=None


In [40]:
c.backward()

In [41]:
print(f'{a.grad=}, {b.grad=}')

a.grad=tensor(2.), b.grad=tensor(0.3536)


**Example 2: Chain rule**

$$
x = 2.0 \\
y = 3.0 \\
z = x \cdot y \\
w = z^2
$$

$$
\frac{\partial z}{\partial x} = y = 3.0 \\
\frac{\partial z}{\partial y} = x = 2.0 \\ 
\frac{\partial w}{\partial z} = 2z = 12.0 \\ 
$$

$$
\frac{\partial w}{\partial x} = \frac{\partial w}{\partial z} \cdot \frac{\partial z}{\partial x} = 12.0 \cdot 3.0 = 36.0 \\
\frac{\partial w}{\partial y} = \frac{\partial w}{\partial z} \cdot \frac{\partial z}{\partial y} = 12.0 \cdot 2.0 = 24.0
$$


In [43]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)
z = x * y
z.retain_grad()
w = z ** 2
print(f'{x.grad=}, {y.grad=}')
w.backward()
print(f'{x.grad=}, {y.grad=}')

x.grad=None, y.grad=None
x.grad=tensor(36.), y.grad=tensor(24.)


In [44]:
z.grad

tensor(12.)

**Example 3**

In [82]:
torch.manual_seed(42)
X = torch.randn(3, 2)
W = torch.randn(2, 1, requires_grad=True)
mm = X @ W
m = mm.mean()
m.backward()

In [83]:
W.grad

tensor([[-0.1839],
        [ 0.0576]])

## **How PyTorch Computes Gradients Automatically**
PyTorch tracks operations on tensors that have `requires_grad=True` and builds a **dynamic computational graph**. During **backpropagation**, it uses the **chain rule** to compute gradients.

### **Computational Graph in PyTorch**
Whenever you perform operations on a tensor with `requires_grad=True`, PyTorch constructs a **computational graph**, where:
- **Nodes** represent tensors.
- **Edges** represent operations that transform tensors.

For example:
```python
import torch

x = torch.tensor(2.0, requires_grad=True)  
y = x ** 2  
z = y + 3  
z.backward()  

print(x.grad)  # This prints 4.0 (since dz/dx = 2x = 2*2 = 4)
```
### **What Happens Here?**
1. **Forward Pass:**
   - $ y = x^2 $  
   - $ z = y + 3 $
2. **Backward Pass:**
   - PyTorch computes **$ \frac{dz}{dx} $** using the **chain rule**:  
     $$
     \frac{dz}{dx} = \frac{dz}{dy} \cdot \frac{dy}{dx} = 1 \times 2x = 2x
     $$
   - Since $ x = 2 $, we get $ x.grad = 4.0 $.

---

### **Chain Rule & Backpropagation**
PyTorch applies the **chain rule** recursively:

$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \cdot \frac{\partial y}{\partial x}
$$

Example with multiple operations:
```python
x = torch.tensor(2.0, requires_grad=True)
y = x ** 3
z = y + x
w = 2 * z
w.backward()  # Compute gradients

print(x.grad)  # 14.0
```
**Gradient Calculation:**
1. $ y = x^3 $, so $ \frac{dy}{dx} = 3x^2 = 3(2^2) = 12 $
2. $ z = y + x $, so $ \frac{dz}{dx} = \frac{dy}{dx} + 1 = 12 + 1 = 13 $
3. $ w = 2z $, so $ \frac{dw}{dz} = 2 $
4. Using the chain rule:
   $$
   \frac{dw}{dx} = 2 \cdot 13 = 26
   $$
   
So, `x.grad = 26`.

---

### **`backward()` & Computational Graph Cleanup**
PyTorch automatically frees the computational graph after calling `.backward()` to **save memory**.  
- To **retain** gradients for multiple backward passes, use `retain_graph=True`:
  ```python
  w.backward(retain_graph=True)
  ```

---

### **`torch.no_grad()` for Disabling Gradients**
If you don’t need gradients (e.g., during inference), use:
```python
with torch.no_grad():
    y = x ** 2  # No gradient tracking
```
or:
```python
x.requires_grad_(False)  # Disables tracking for x
```

---

### **`torch.autograd.grad()` for Manual Gradients**
Instead of calling `.backward()`, you can compute gradients manually:
```python
grad = torch.autograd.grad(w, x)  # Computes dw/dx
print(grad)  # (tensor(26.),)
```

---

### **Summary**
- PyTorch builds a **computational graph** dynamically.  
- Uses **backpropagation** & **chain rule** to compute gradients.  
- `.backward()` propagates gradients automatically.  
- Use `torch.no_grad()` for inference to save memory.  
- Use `torch.autograd.grad()` for manual gradient computations.

---

# **Building an Autograd Engine from Scratch**

Let’s build a **basic autograd engine from scratch** similar to Andrej Karpathy's `micrograd`. This will help us understand how PyTorch’s **autograd** system works internally.

We will create a minimal **automatic differentiation engine** that supports:
- Scalar operations  
- **Computational graphs**  
- **Backward propagation** using the **chain rule**  


## **Step 1: Define a Computational Graph Node**
Each value in our computation graph needs to:
- Store its **current value**.
- Keep track of the **operation** that produced it.
- Store **gradients** during backpropagation.

We define a **Value class**:

```python
import numpy as np

class Value:
    def __init__(self, data, _children=(), _op=""):
        self.data = data  # Scalar value
        self.grad = 0  # Gradient (initialized to 0)
        self._backward = lambda: None  # Backward function
        self._children = set(_children)  # Track parent nodes
        self._op = _op  # Operation that produced this value

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"
```


## **Step 2: Implement Basic Operations**
We define **addition, multiplication, and power** with proper **gradient computation**.

### **Addition**
For **$ z = x + y $**, we apply:
$$
\frac{dz}{dx} = 1, \quad \frac{dz}{dy} = 1
$$
```python
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)  # Support numbers
        out = Value(self.data + other.data, (self, other), "+")

        def _backward():
            self.grad += out.grad  # dz/dx = 1
            other.grad += out.grad  # dz/dy = 1

        out._backward = _backward
        return out
```


### **Multiplication**
For **$ z = x \cdot y $**, we apply:
$$
\frac{dz}{dx} = y, \quad \frac{dz}{dy} = x
$$
```python
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), "*")

        def _backward():
            self.grad += other.data * out.grad  # dz/dx = y
            other.grad += self.data * out.grad  # dz/dy = x

        out._backward = _backward
        return out
```


### **Power Function**
For **$ z = x^n $**, we apply:
$$
\frac{dz}{dx} = n \cdot x^{n-1}
$$
```python
    def __pow__(self, exponent):
        assert isinstance(exponent, (int, float)), "Only supports int or float exponent"
        out = Value(self.data ** exponent, (self,), f"**{exponent}")

        def _backward():
            self.grad += exponent * (self.data ** (exponent - 1)) * out.grad

        out._backward = _backward
        return out
```

---

## **Step 3: Implement `backward()` for Backpropagation**
To compute gradients, we use **reverse-mode autodiff**:
- Start from the **final node** (i.e., loss).
- Recursively apply **the chain rule** using `_backward()`.
- Traverse nodes **in topological order**.

```python
    def backward(self):
        # Topological sorting using DFS
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._children:
                    build_topo(child)
                topo.append(v)  # Append after children

        build_topo(self)

        # Initialize gradient at the final node (output)
        self.grad = 1

        # Traverse in reverse order and apply chain rule
        for v in reversed(topo):
            v._backward()
```


## **Step 4: Test the Autograd Engine**
Now, let's test our **manual autograd engine** by computing derivatives.

### **Example: Compute $ y = x^2 + 2x + 1 $ and its derivative**
$$
\frac{dy}{dx} = 2x + 2
$$

```python
x = Value(3.0)  # Initialize variable
y = x**2 + 2*x + 1  # Compute function
y.backward()  # Compute gradients

print(f"x: {x.data}, y: {y.data}, dy/dx: {x.grad}")
```
#### **Expected Output:**
```
x: 3.0, y: 16.0, dy/dx: 8.0
```
since:
$$
\frac{dy}{dx} = 2(3) + 2 = 8
$$